In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
isNotebook = '__file__' not in locals()

In [3]:
TEST = isNotebook

In [4]:
import twinews.config as tConf
tConf.mongoLocation = 'octods'

In [5]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.evaluation import metrics
from twinews.evaluation.utils import *
import time
import pymongo

In [ ]:
logger = Logger(tmpDir('logs') + "/watch-rankings.log") if isNotebook else Logger("watch-rankings.log")

In [ ]:
# We get scores collection and the rankings GridFS:
twinewsScores = getTwinewsScores()
twinewsRankings = getTwinewsRankings()

In [6]:
dataV1 = getEvalData(1)
bp(dataV1)
dataV2 = getEvalData(2)
bp(dataV2)

--> tic: 8.63s | message: Eval data loaded
--> toc total duration: 12.91s | message: Got Twinews evaluation data
{
  candidates: 
  {
    100022528: 
    [
      {
        http://a.msn.com/00/en-us/AAuAIog?ocid=st,
        http://a.msn.com/01/en-us/AAv6Yoa?ocid=st,
        ...,
        https://www.zerohedge.com/news/2018-01-31/earths-magnetic-field-shifting-poles-may-flip-could-get-ba,
        https://ziffblog.wordpress.com/2018/01/26/stare-decisis-and-the-consubstantiality-of-the-washington-
      }
    ],
    100024324: 
    [
      {
        http://13wham.com/news/local/st-john-fisher-responds-after-inappropriate-fliers-found-on-campus,
        http://a.msn.com/05/en-us/BBJ66v8?ocid=st,
        ...,
        https://www.yahoo.com/news/brash-ex-milwaukee-sheriff-david-clarke-faces-civil-161831030--politics.h,
        https://www.zerohedge.com/news/2018-02-07/yuan-crashing-after-huge-china-trade-surprise
      }
    ],
    100064338: 
    [
      {
        http://a.msn.com/0C/en-us/AAu

--> tic: 6.53s | message: Eval data loaded
--> toc total duration: 9.76s | message: Got Twinews evaluation data
{
  candidates: 
  {
    100022528: 
    [
      {
        http://a.msn.com/0A/en-us/BBI0fMo?ocid=st,
        http://abc27.com/2018/01/05/a-basketball-life-donyae-baylor-carroll/,
        ...,
        https://www.yorkshireeveningpost.co.uk/news/crime/how-many-clare-s-law-requests-are-approved-by-west,
        https://www.zerohedge.com/news/2017-12-28/global-stocks-rally-new-record-highs-dollar-tumbles
      }
    ],
    100024324: 
    [
      {
        http://a.msn.com/0A/en-us/BBI0fMo?ocid=st,
        http://a.msn.com/0B/en-us/BBIb6wL?ocid=st,
        ...,
        https://www.yahoo.com/news/fallout-louisiana-teachers-arrest-rages-035613133.html?.tsrc=fauxdal,
        https://www.yorkregion.com/whatson-story/7998261-full-circle-a-caffeine-fuelled-record-breaking-conc
      }
    ],
    100064338: 
    [
      {
        http://a.msn.com/00/en-us/BBHyFMw?ocid=st,
        http:

In [8]:
serialize(dataV1, homeDir() + "/test.pickle")

In [9]:
serialize(dataV2, homeDir() + "/validation.pickle")

In [10]:
print(dataV1["meta"])

{'usersCount': 21239, 'trainNewsCount': 323572, 'testNewsCount': 138785, 'totalNewsAvailable': 570210, 'trainMeanNewsPerUser': 28.0, 'testMeanNewsPerUser': 10.67, 'trainMinNewsPerUser': 8, 'testMinNewsPerUser': 2, 'trainMaxNewsPerUser': 443, 'testMaxNewsPerUser': 164, 'created': '2020.03.24-14.13.25', 'ranksLength': 1000, 'splitDate': '2018-01-15', 'startDate': '2017-10-01', 'endDate': '2018-02-16', 'id': 1}


# Deletions

In [ ]:
# 11	lda-d4f1d	cosine	False	30	gensim-lda	1	300000	60	0.0005	100	False	0.41897
# del twinewsRankings['lda-d4f1d']

In [ ]:
# nmf-9cd4f	False	30	100000	200	100	nndsvd	0	0.49322
# del twinewsRankings['nmf-9cd4f']

# Watching for rankings having no distance scores

In [ ]:
keys = twinewsRankings.keys()
bp(keys)
noScoresWithCandidates = set()

In [ ]:
for key in pb(keys, logger=logger, printRatio=0.001):
    rankings = twinewsRankings[key]
    if rankings is None or len(rankings) == 0:
        log(key + " is empty.", logger)
    else:
        element = rankings[list(rankings.keys())[0]][0][0]
        if isinstance(element, str):
            log(key + " has only items.", logger)
            noScoresWithCandidates.add(key)
        elif isinstance(element, tuple):
            log(key + " has tuples.", logger)
        else:
            log(key + " WTF.", logger)

In [ ]:
log(noScoresWithCandidates, logger)

### Result

```python
{'lda-64d0c', 'dbert-ft-2529b', 'tfidf-dce5a', 'tfidf-44aea', 'combin-abe90', 'dbert-ft-833d3', 'lda-2fa8e', 'lda-d1220', 'nmf-6d480', 'ideal-5ac2d', 'lda-de387', 'combin-7548f', 'nmf-054a5', 'combin-5f39b', 'worst-f3f99', 'tfidf-ae6af', 'random-a1ece', 'lda-759de', 'lda-71511', 'nmf-9cd4f', 'combin-29a8c', 'random-e27ab', 'combin-2122b', 'combin-945df', 'dbert-ft-3ce99', 'random-21e0b', 'combin-be9fe', 'dbert-ft-3a57f', 'combin-a3547', 'BM25_yf-78651', 'lda-186ce', 'combin-3b2ee', 'bm25-1a0ad', 'nmf-dea50', 'combin-4f69f', 'nmf-ee6c6', 'bm25-6da22', 'nmf-84f4b', 'bm25-1dab5', 'combin-2622b', 'bm25-e2365', 'combin-d19aa', 'nmf-09f9c', 'combin-63c3f', 'combin-3f805', 'combin-9c12b', 'random-f03b8', 'combin-d536d', 'combin-80686', 'bm25-c84fd', 'dbert-ft-6c7f3', 'nmf-87941', 'lda-b637b', 'lda-d4f1d', 'combin-37711', 'combin-56b83', 'combin-3016b', 'combin-7e45c', 'random-d87ed', 'combin-86cb0', 'combin-aa56c', 'bm25-9eee2', 'combin-f7cfd', 'dbert-ft-a259b', 'lda-b123f', 'dbert-ft-71ce1', 'bm25-c75d0', 'nmf-b286d', 'combin-e515d', 'nmf-e51ed', 'nmf-ad87b', 'lda-d3b82', 'combin-52a29', 'dbert-ft-cb33b', 'combin-7ba13', 'tfidf-40a9b', 'combin-4dad1', 'combin-91932', 'random-a9655', 'nmf-330c8', 'bm25-abfe4', 'random-6fae4', 'combin-ceded', 'combin-87c28', 'tfidf-71e3d', 'combin-52394', 'dbert-ft-11978', 'combin-17c18', 'dbert-ft-6895d', 'combin-332f8', 'bm25-a34a0', 'nmf-b61b9', 'combin-47edf', 'lda-94782', 'tfidf-73155', 'lda-7786b', 'nmf-17852', 'bm25-a91ce', 'lda-b849e', 'combin-cb389', 'combin-8f3f8', 'bm25-dedc0', 'bm25-0c4d5', 'combin-0d4be', 'combin-55d93', 'lda-af807', 'dbert-ft-fdc3b', 'random-04de9', 'combin-ed086', 'nmf-ef428', 'lda-51ce2', 'combin-efa9e', 'combin-9267b', 'bm25-7efac', 'combin-e2a5a', 'combin-08a39', 'combin-0df7b', 'lda-aa8d5', 'nmf-51495', 'nmf-d9bba', 'dbert-ft-c3c19', 'BM25_yf-42d08', 'dbert-ft-d66a1', 'random-093c0', 'tfidf-8a99c', 'combin-31b05', 'combin-481bb', 'dbert-ft-9cd08', 'nmf-b1354', 'random-4d05f', 'combin-e14e5', 'tfidf-6f201', 'nmf-2d0d6', 'lda-34076'}
```